In [9]:
#импорт необходимых библиотек

import numpy as np
import pandas as pd
import os
import time
import datetime
from random import randint, seed

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

from core_changed import Backtest, PlotPoint, IStrategy
from multiprocessing import Pool
from math import log
from datetime import datetime

from tslearn.metrics import dtw
import itertools
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import MinMaxScaler

## Кластеризация и поиск оптимальных кластеров

In [10]:
total_path = 'C:\\Users\\Алексей\\Учеба\\ЦМФ\\Проекты\\Алготрейдинг\\Задание 6'
data_path = total_path + '\\stock_data'

In [11]:
def calc_centers_of_clusters(df,labels):
    # функция рассчета центров кластеров
    X = df.copy()
    X ['labels'] = labels
    numbers = list(set(labels))
    result = {}
    for i in numbers:
        cluster_data = X[X['labels']==i].copy()
        sum_dict = dict(cluster_data.mean())
        del sum_dict['labels']
        result[i] = sum_dict
        
    return result

In [12]:
def calc_RMSSTD(df,labels):
    # функция рассчета компактности кластеров
    
    centers_of_clusters = calc_centers_of_clusters(df,labels)
    X = df.copy()
    X ['labels'] = labels

    total_nominator = 0
    total_denominator = 0
    for i in centers_of_clusters:
        cluster_center = centers_of_clusters[i]
        cluster_data = X[X['labels']==i].copy()
        cluster_data.drop('labels', axis=1, inplace=True)
        diff = np.power(cluster_data - cluster_center, 2)
        norm_diff = diff.sum(axis = 1)
        sum_el = sum(norm_diff)
        total_nominator= total_nominator + sum_el

        n_el = cluster_data.shape[0]
        total_denominator= total_denominator + n_el-1

    n_features = cluster_data.shape[1]
    RMSSTD = (total_nominator/(n_features*total_denominator))**0.5
    
    return RMSSTD

In [13]:
def find_best_param(d, eps_list, min_samples_list, flag):
    # кастомная функция поиска оптимальных параметров по значениям метрик
    # Принимает: словарь с метриками, список eps, список min_samples, флаг, определяющий метрику
    # возвращает: наилучши
    best_eps = np.nan
    best_min_samples = np.nan
    if flag == 's' or flag == 'c-h':
        best_score = -2
        for i in d:
            eps = i
            sub_list = d[i]
            for j in sub_list:
                if j > best_score:
                    best_eps = eps
                    best_min_samples = min_samples_list[sub_list.index(j)]
                    best_score = j
        if flag == 's':
            print('Лучший силуэт: '+str(best_score))
        else:
            print('Лучший индекс Калински-Харабаша: '+str(best_score))
        print('Eps при нем: '+str(best_eps))
        print('Min_samples при нем: '+str(best_min_samples))
        
        
    elif flag == 'd-b':
        best_score = 100000
        for i in d:
            eps = i
            sub_list = d[i]
            for j in sub_list:
                if j < best_score:
                    best_eps = eps
                    best_min_samples = min_samples_list[sub_list.index(j)]
                    best_score = j
        print('Лучший индекс Дэвиса-Болдуина: '+str(best_score))
        print('Eps при нем: '+str(best_eps))
        print('Min_samples при нем: '+str(best_min_samples))
        
        
    return best_eps, best_min_samples, best_score

In [14]:
total_df = pd.read_csv('data_final_v1_upd.csv')

In [15]:
## пересобираем датасет

new_col_name = list(total_df['Datetime'])
del total_df['Datetime']



total_df = total_df.T
total_df = total_df.reset_index()
total_df = pd.DataFrame(np.array(total_df), columns = ['ticker'] + new_col_name)

feature_columns = list(total_df.columns)[1:]

tickers = list(total_df['ticker'])

### 1. Делим на трейн и тест

In [16]:
test_size = 0.3

In [17]:
test_columns = ['ticker']+feature_columns[(-1)*int(np.round(len(total_df.columns)*test_size)):]
train_columns = ['ticker']+feature_columns[:(-1)*int(np.round(len(total_df.columns)*test_size))]

In [18]:
test_df = total_df[test_columns]
train_df = total_df[train_columns]

In [19]:
test_df.to_csv('test.csv',index = False)
train_df.to_csv('train.csv',index = False)

### 2. Нормализуем и рассчитываем матрицу dtw расстояний между объектами

In [20]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [21]:
tickers = list(train_df['ticker'])
data_columns = list(train_df.columns)[1:]

tickers_pairs= list(itertools.combinations(tickers, 2))

train_norm_df = pd.DataFrame(columns = ['ticker']+data_columns)

In [22]:
#нормализация
for i in tickers:
    ts = np.array(train_df[data_columns][train_df['ticker']==i]).reshape(-1,1)
    scaler = MinMaxScaler()
    ts_norm = scaler.fit_transform(np.array(ts))
    ts_norm = list(ts_norm.reshape(393,))

    new_row = {data_columns[i]:ts_norm[i] for i in range(0,len(ts_norm))}
    new_row['ticker'] = i
    train_norm_df = train_norm_df.append(new_row, ignore_index=True)

In [23]:
feature_train_columns = list(train_norm_df.columns)[1:]

In [24]:
train_norm_df

,ticker,2017-11-08 00:00:00,2017-11-09 00:00:00,2018-01-08 00:00:00,2018-01-09 00:00:00,2018-01-10 00:00:00,2018-03-15 00:00:00,2018-03-16 00:00:00,2018-04-12 00:00:00,2018-04-17 00:00:00,...,2021-05-14 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-31 00:00:00
0,1.0,0.206554,0.218256,0.255120,0.262142,0.284962,0.181978,0.177882,0.170860,0.152721,...,0.901112,0.917496,0.935050,0.917496,0.930369,0.911059,1.000000,0.987127,0.970158,0.952604
1,2.0,0.434615,0.436538,0.969872,0.960256,0.949359,0.756410,0.754487,0.704487,0.580769,...,0.587821,0.591026,0.601923,0.568590,0.581410,0.555769,0.594872,0.593590,0.580128,0.564103
2,5.0,0.986395,1.000000,0.904762,0.918367,0.887755,0.775510,0.768707,0.670068,0.636054,...,0.091837,0.057823,0.037415,0.037415,0.006803,0.000000,0.017007,0.040816,0.027211,0.013605
3,9.0,0.468031,0.465473,0.413043,0.409207,0.397698,0.278772,0.290281,0.278772,0.254476,...,0.887468,0.878517,0.891304,0.927110,0.870844,0.863171,0.868286,0.859335,0.875959,0.927110
4,10.0,0.991045,0.976119,1.000000,0.943284,0.931343,0.579104,0.597015,0.349254,0.211940,...,0.370149,0.337313,0.334328,0.343284,0.352239,0.379104,0.373134,0.367164,0.349254,0.358209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,603987.0,0.264335,0.261082,0.132981,0.151688,0.144774,0.114274,0.115087,0.154534,0.124034,...,0.785685,0.827572,0.813339,0.793819,0.788939,0.741765,0.768605,0.795445,0.891013,1.000000
1802,603989.0,0.761845,0.793641,0.731920,0.778678,0.969451,0.656484,0.683292,0.769327,0.744389,...,0.775561,0.720698,0.746259,0.711347,0.700748,0.712594,0.738778,0.799252,0.814214,0.832294
1803,603991.0,0.480010,0.489274,0.306192,0.306192,0.291565,0.185032,0.180156,0.283764,0.276938,...,0.939542,0.941736,0.928084,0.889078,0.866650,0.864700,0.900049,0.945880,0.963676,1.000000
1804,603993.0,0.571669,0.564924,0.635750,0.625632,0.620573,1.000000,0.961214,0.797639,0.694772,...,0.414840,0.443508,0.455312,0.460371,0.408094,0.424958,0.396290,0.389545,0.411467,0.423272


In [26]:
%%time
# считаем пары расстояний
dist_dict = {}
for i in tickers_pairs:
    print(tickers_pairs.index(i))
    dist = dtw(list(np.array(train_norm_df[train_norm_df['ticker']==i[0]])[0][1:]), \
               list(np.array(train_norm_df[train_norm_df['ticker']==i[1]])[0][1:]))
    dist_dict[i]=dist

In [177]:
# собираем матрицу расстояний
keys = list(dist_dict.keys())
values = list(dist_dict.values())

dist_matrix = np.zeros((len(tickers),len(tickers)))
dist_matrix[:] = np.nan
dist_matrix = pd.DataFrame(dist_matrix, columns = tickers)
dist_matrix.index = tickers

for i in range(0,len(keys)):
    dist_matrix.loc[keys[i][0],keys[i][1]] = values[i]
    dist_matrix.loc[keys[i][1],keys[i][0]] = values[i]
    
for i in range(0,len(tickers)): 
    dist_matrix.loc[tickers[i],tickers[i]] = 0

In [179]:
dist_matrix.to_csv('dtw_matrix.csv')

#### грузим матрицу и кластеризуем

In [29]:
dist_matrix = pd.read_csv('dtw_matrix.csv')
dist_matrix.index = dist_matrix['Unnamed: 0']

del dist_matrix['Unnamed: 0']

In [30]:
eps_list = list(np.arange(1,15,0.25))
min_samples_list = list(np.arange(5,30,1))

In [32]:
%%time
# кластеризуем и сразу считаем метрики
silhouette_dict = {}
davies_bouldin_dict = {}
calinski_harabasz_dict = {}

for i in eps_list:
    silhouette_list =list()
    davies_bouldin_list =list()
    calinski_harabasz_list =list() 
    for j in min_samples_list:
        clustering = DBSCAN(eps=i, min_samples=j, metric = 'precomputed').fit(dist_matrix)
        labels = clustering.labels_
        
        if (len(set(labels))==1 or len(set(labels))==len(labels)):
                    silhouette_list.append(-1.5)
                    davies_bouldin_list.append(1000)
                    calinski_harabasz_list.append(-1.5)
        else:
            silhouette_list.append(silhouette_score(train_norm_df[feature_train_columns], labels))
            davies_bouldin_list.append(davies_bouldin_score(train_norm_df[feature_train_columns], labels))
            calinski_harabasz_list.append(calinski_harabasz_score(train_norm_df[feature_train_columns], labels))
            
    silhouette_dict[i] = silhouette_list
    davies_bouldin_dict[i] = davies_bouldin_list 
    calinski_harabasz_dict[i] = calinski_harabasz_list

In [18]:
find_best_param(silhouette_dict, eps_list, min_samples_list, 's')

Лучший силуэт: 0.1560875628717324
Eps при нем: 1.5
Min_samples при нем: 5


(1.5, 5, 0.1560875628717324)

In [19]:
find_best_param(calinski_harabasz_dict, eps_list, min_samples_list, 'c-h')

Лучший индекс Калински-Харабаша: 131.0139691474577
Eps при нем: 1.0
Min_samples при нем: 26


(1.0, 26, 131.0139691474577)

In [20]:
find_best_param(davies_bouldin_dict, eps_list, min_samples_list, 'd-b')

Лучший индекс Дэвиса-Болдуина: 2.3723221055523713
Eps при нем: 1.5
Min_samples при нем: 5


(1.5, 5, 2.3723221055523713)

#### результат: кластеризация при лучших параметрах. Глобально - все скоры плохие. Возьмем случай с наилучшим силуэтом

In [58]:
e = 1.5
min_s = 5

clustering = DBSCAN(eps=e, min_samples=min_s, metric = 'precomputed').fit(dist_matrix)
labels = clustering.labels_

print('Число кластеров '+str(len(set(labels))))

Число кластеров 2


состав этих кластеров

In [59]:
с = Counter(labels)
с

Counter({0: 1801, -1: 5})

все очень плохо. все данные слились в 1 кластер, второй кластер - шум.

## 3. Альтернативный подход.
#### Подойдем к задаче альтернативно. Найдем параметры, при которых число кластеров наибольшее, а затем выбрем несколько самых компактных из них.

In [122]:
eps_list = list(np.arange(0.1,1,0.01))
min_samples_list = list(np.arange(2,4,1))

In [123]:
%%time
# кластеризуем и сразу считаем метрики
n_clusters_dict = {}
davies_bouldin_dict = {}
calinski_harabasz_dict = {}

for i in eps_list:
    n_clusters_list =list()
    for j in min_samples_list:
        clustering = DBSCAN(eps=i, min_samples=j, metric = 'precomputed').fit(dist_matrix)
        labels = clustering.labels_
        n_clusters_list.append(len(set(labels)))
            
    n_clusters_dict[i] = n_clusters_list

CPU times: total: 9.36 s
Wall time: 9.37 s


In [132]:
n_clusters_dict

{0.1: [1, 1],
 0.11: [1, 1],
 0.12: [1, 1],
 0.13: [1, 1],
 0.13999999999999999: [1, 1],
 0.14999999999999997: [1, 1],
 0.15999999999999998: [1, 1],
 0.16999999999999998: [1, 1],
 0.17999999999999997: [1, 1],
 0.18999999999999995: [1, 1],
 0.19999999999999996: [1, 1],
 0.20999999999999996: [1, 1],
 0.21999999999999995: [1, 1],
 0.22999999999999995: [1, 1],
 0.23999999999999994: [1, 1],
 0.24999999999999992: [1, 1],
 0.2599999999999999: [1, 1],
 0.2699999999999999: [1, 1],
 0.2799999999999999: [1, 1],
 0.2899999999999999: [1, 1],
 0.29999999999999993: [1, 1],
 0.30999999999999994: [1, 1],
 0.3199999999999999: [1, 1],
 0.32999999999999985: [1, 1],
 0.33999999999999986: [1, 1],
 0.34999999999999987: [1, 1],
 0.3599999999999999: [2, 2],
 0.3699999999999999: [3, 2],
 0.3799999999999999: [4, 2],
 0.3899999999999999: [7, 2],
 0.3999999999999998: [7, 3],
 0.4099999999999998: [6, 4],
 0.4199999999999998: [6, 5],
 0.4299999999999998: [5, 5],
 0.43999999999999984: [10, 4],
 0.44999999999999984: [

In [153]:
# наилучше параметры (поиск опущен)
e = 0.85
min_s = 2

clustering = DBSCAN(eps=e, min_samples=min_s, metric = 'precomputed').fit(dist_matrix)
labels = clustering.labels_

print('Число кластеров '+str(len(set(labels))))

Число кластеров 30


In [184]:
train_norm_df_2 = train_norm_df.copy()
train_norm_df_2['label'] = labels 

sub_df = train_norm_df_2[train_norm_df_2['label'].isin(actual_labels)]

#### для каждого кластера считаем компактность, берем кластера с компактностью менее metic_border

In [235]:
metic_border = 0.11

In [188]:
metic_dict = {}
for i in set(list(train_norm_df_2['label'])):
    sub_df = train_norm_df_2[train_norm_df_2['label']==i]
    metric = calc_RMSSTD(sub_df[feature_train_columns],sub_df['label'])
    metic_dict[i] = np.round(metric,2)

In [236]:
good_clusters = []

for i in metic_dict:
    if metic_dict[i] < metic_border:
        good_clusters.append(i)

In [237]:
good_clusters

[1, 2, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 20, 21, 23, 24, 25, 26, 27, 28]

In [238]:
train_df_2 = train_df.copy()
train_df_2['label'] = labels
train_df_2['good_cluster'] = 0
train_df_2.loc[train_df_2['label'].isin(good_clusters),'good_cluster'] = 1

In [239]:
train_df_2.to_csv('train_and_clusters.csv', index = False)

далее попробуем протестировать эти результаты на стратегии